In [ ]:
!pip install langchain langchain_openai langchain_community pypdf faiss-cpu python-dotenv



Das Bereitstellen von Daten nennt man gerne _RAG_ oder _Retrieval Augmented Generation_. Der "Retrieval" Part sagt aus, dass man zu der Fragestellung oder Aufgabenstellung die relevanten Informationen wie z.B. Textstellen aus Dokumenten holt oder sucht.
Diese Informationen werden mitsamt der Fragestellung im Prompt mitgegeben (augmented). Die KI generiert daraus eine Antwort. So kann die KI auf Basis von ihr bis dahin unbekannter Daten, Antworten generieren.

Aber wie findet man die Textstellen? Dazu nutzt man im einfachsten Fall eine Suchfunktion. Da jedoch Suchfunktionen, basierend auf Schlüsselwortsuche, wenig vielversprechend sind, nutzt man meist sogenannte Embeddings. Das sind numerische Repräsentationen von Texten, die auch den "Sinn" oder die Semantik beinhalten. So kann man Texte finden, die auf irgendeine Weise relevant oder semantisch ähnlich ist, wie ein anderer Text. Bei diesem anderen Text handelt es sich dann um die Frage, die man beantwortet haben möchte. Die Embeddings werden in einer Vektordatenbank gespeichert. Diese Datenbank bietet uns dann die Suchfunktion, die wir benötigen.

Das Thema RAG kann fast beliebig kompliziert werden. Deswegen beschränken wir uns auf die einfachste Umsetzung, mit einer In-Memory-Vektordatenbank.

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter

documents = PyPDFLoader("eine-ki-geschichte.pdf").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(documents)


vectorstore = FAISS.from_documents(
    docs,
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Beantworte die Fragen auf Basis des folgenden Textes:
{context}

Frage: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("Welche Mission hat die KI in der Geschichte?")